In [14]:
import cv2
import urllib
import numpy as np
from efficientnet_pytorch import EfficientNet
import torch
from torchvision import transforms
from PIL import Image


In [49]:
def predict(file, file_type="url", category=""): # category可以不給
    
    index2label = {
    0: "abstract",
    1: "animal print",
    2: "camouflage",
    3: "floral",
    4: "geometric",
    5: "ikat",
    6: "melange",
    7: "placement",
    8: "plaids",
    9: "solid",
    10: "spots",
    11: "strips",
    }
    
    def url_to_image_original(url, width=200, height=200):
        resp = urllib.request.urlopen(url)
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR) 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image

    face_cascade = cv2.CascadeClassifier('C:\\Users\\andyy\\anaconda3\\envs\\python38\\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_default.xml')

    def has_face(image):  # 判斷是否有人臉
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY )
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1 , minNeighbors=5, minSize=(int(gray.shape[0]/15),int(gray.shape[1]/20)))
        def get_largest_part(a_list):  # 回傳最大的區塊
            if len(a_list)<2:
                return a_list
            else:
                return sorted(a_list, key=lambda x: x[2]*x[3], reverse=True)[:1]
        faces = get_largest_part(faces)
        if faces == ():
            return False  # 如果沒有偵測到，或是偵測出來位置太低就回傳否
        elif faces[0][1] > image.shape[0] * 0.3:
            return False
        else:
            return True
        
    def cut_image(cur_image, original_clothing):  # 切割衣服
        upper_clothing = ["Short Sleeve", "Softshells", "Long Sleeve", "Windwear", "Hoodie & Sweatshirt", "Jackets & Vests",
                         "Tank", "Tops", "Sports Bra", "Polo", "Vest", "Insulated & Down", "Fleece", "Shirt"]
        lower_clothing = ["Shorts","Pants", "Leggings & Tights", "Bottoms"]
        lower_all_clothing = ["Skirt & Dresses"]
        upper_all_clothing = ["Surfing", "Rainwear", "WetSuit", "Weather Protection", "Springsuit"]
        if bool(set(original_clothing) & set(upper_clothing)):  #上半身情況
            if has_face(cur_image):
                y_start_coefficient = 0.3
                height_coefficient = 0.5
            else:
                y_start_coefficient = 0.2
                height_coefficient = 0.6
            clothing_condition = 0
        elif bool(set(original_clothing) & set(lower_clothing)):  #下半身情況
            if has_face(cur_image):
                y_start_coefficient = 0.4
                height_coefficient = 0.5
            else:
                y_start_coefficient = 0.15
                height_coefficient = 0.65
            clothing_condition = 1
        elif bool(set(original_clothing) & set(lower_all_clothing)):  #下半身全身都有可能情況
            if has_face(cur_image):
                y_start_coefficient = 0.25
                height_coefficient = 0.55
            else:
                y_start_coefficient = 0.2
                height_coefficient = 0.6
            clothing_condition = 2
        elif bool(set(original_clothing) & set(upper_all_clothing)):  #上半身全身都有可能情況
            if has_face(cur_image):
                y_start_coefficient = 0.3
                height_coefficient = 0.5
            else:
                y_start_coefficient = 0.2
                height_coefficient = 0.6
            clothing_condition = 3
        else:
            y_start_coefficient = 0.2
            height_coefficient = 0.6
            clothing_condition = 4
        
        x_start = int(0.2 * cur_image.shape[1])
        y_start = int(y_start_coefficient * cur_image.shape[0])
        # 裁切區域的長度與寬度
        width = int(0.6 * cur_image.shape[1])
        height = int(height_coefficient * cur_image.shape[0])
        # 裁切圖片
        crop_img = cur_image[y_start:y_start+height, x_start:x_start+width]
        # 裁切完再resize
        train_images_cur = cv2.resize(crop_img, (256, 256))
        return train_images_cur, clothing_condition



    
    if file_type == "url":
        image = url_to_image_original(file)
    elif file_type == "file_name":
        image = Image.open(file)
        image = np.array(image)
    else:
        print("Wrong file type.")
        return None
        
    img_ = np.array(image)
    if img_.ndim != 3:
        print("dimension error")
        return None
    if img_.shape[2] == 4:  # 處理有 Transparency (RGBA) 的情況
        img = img.convert('RGB')
        img_ = np.array(img)
    image, clothing_condition = cut_image(img_, category) 
    
#     import matplotlib.pyplot as plt 
#     print(image.shape)
#     plt.imshow(image)
#     plt.show()
    
    
    datagen_valid_test = transforms.Compose([
    transforms.ToTensor(),
    ])
    image = datagen_valid_test(Image.fromarray(image))
    image = torch.unsqueeze(image, 0)
    
    model = EfficientNet.from_name('efficientnet-b1')
    model.load_state_dict(torch.load('./models/efficientnet.h5', map_location={'cuda:0': 'cpu'}))
    with torch.no_grad():
        model.eval()
        inputs = image.float()  # 因為train有經過資料增強，已是float型態，valid、test也必須轉成float型態
        outputs = model(inputs)
        outputs = torch.argmax(outputs, 1)
        
        return index2label[outputs.item()]
    
    
    
print("example: url + category")    
output = predict("https://bilab.synology.me/smis/brand/reebok/reebok_kxxojml3.png", "url", "Hoodie & Sweatshirt")
print(output)
print("example: url + without category")    
output = predict("https://bilab.synology.me/smis/brand/reebok/reebok_kxxojml3.png", "url")
print(output)
print("example: file + category")
output = predict("./batch1_images_cut_formal/train/abstract/adidas__hjvbsam.png", "file_name", "Hoodie & Sweatshirt")
print(output)
print("example: file + without category")
output = predict("./batch1_images_cut_formal/train/abstract/adidas__hjvbsam.png", "file_name")
print(output)

example: url + category
placement
example: url + without category
placement
example: file + category
abstract
example: file + without category
abstract


### 測試看看 訓練集會不會跳錯(或是直接抓db正確率大概是多少來看看程式有沒有錯)

In [50]:
# # 測試看看 訓練集會不會跳錯(或是直接抓db正確率大概是多少來看看程式有沒有錯)
# import pymongo
# import pandas as pd
# import copy
# myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
# dblist = myclient.list_database_names()
# mydb = myclient["fashionAI"]
# mycol = mydb["Products"]
# mycol
# # for x in mycol.find():
# #     print(x)

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'fashionAI'), 'Products')

In [51]:
# picts_link = []
# patterns = []
# clothings = []

# for x in mycol.find():
# #     print(x.keys())
#     try: 
#         if x["Pattern_v2"] and x["OurImage"] and x["Clothing"]:
#             patterns.append(x["Pattern_v2"][0])
#             picts_link.append(x["OurImage"])
#             clothings.append(x["Clothing"])
#     except:
#         print("item image url %s has no Pattern" % (x["OurImage"],))

item image url https://bilab.synology.me/smis/brand/burton/Burton_2020-01-05_9ijawi5d.png has no Pattern
item image url https://bilab.synology.me/smis/brand/burton/Burton_2020-01-05__9080vz0.png has no Pattern
item image url https://bilab.synology.me/smis/brand/burton/Burton_2020-01-05_pshha_9c.png has no Pattern


In [52]:
# # 建立url pattern 的dataframe
# url_pattern_df_copy = pd.DataFrame({"image_url":picts_link, "pattern":patterns, "clothing": clothings})
# url_pattern_df = copy.deepcopy(url_pattern_df_copy)
# url_pattern_df['image_name'] = url_pattern_df['image_url'].apply(lambda x: x.split('/')[-1].split('.')[0])

In [53]:
# # 刪除非衣服、褲子等非關注的 clothing
# clothing_list = ["Short Sleeve", "Softshells", "Long Sleeve", "Windwear", "Hoodie & Sweatshirt", "Jackets & Vests",
#                  "Tank", "Tops", "Sports Bra", "Polo", "Vest", "Insulated & Down", "Fleece", "Shirt", "Full-Zip,Cape",
#                  "Shorts","Pants", "Leggings & Tights", "Bottoms", "Joggers", "Skirt & Dresses", "Surfing", "Rainwear", "WetSuit",
#                  "Weather Protection", "Springsuit"]

# print("共有 %s 筆資料不是衣服或褲子，因此不納入模型預測中" % (len(url_pattern_df) - len(url_pattern_df[url_pattern_df.apply(lambda x: bool(set(x.clothing) & set(clothing_list)), axis=1)]),))
# url_pattern_df = url_pattern_df[url_pattern_df.apply(lambda x: bool(set(x.clothing) & set(clothing_list)), axis=1)]
# url_pattern_df = url_pattern_df.reset_index(drop=True)
# print("共剩下 %s 筆資料" % (len(url_pattern_df),))
# url_pattern_df.head()


# # 刪除非上面 12 種 pattern
# pattern_list = ["abstract", "animal print", "camouflage", "floral", "geometric", "ikat",
#                  "melange", "placement", "plaids", "solid", "spots", "strips"]

# print("共有 %s 筆資料不是在目標pattern中，因此不納入模型預測中" % (len(url_pattern_df) - len(url_pattern_df[url_pattern_df['pattern'].isin(pattern_list)]),))
# url_pattern_df = url_pattern_df[url_pattern_df['pattern'].isin(pattern_list)]
# url_pattern_df = url_pattern_df.reset_index(drop=True)

# print("共剩下 %s 筆資料" % (len(url_pattern_df),))
# url_pattern_df.head()

共有 400 筆資料不是衣服或褲子，因此不納入模型預測中
共剩下 11148 筆資料
共有 785 筆資料不是在目標pattern中，因此不納入模型預測中
共剩下 10363 筆資料


,image_url,pattern,clothing,image_name
0,https://bilab.synology.me/smis/brand/reebok/re...,placement,[Hoodie & Sweatshirt],reebok_kxxojml3
1,https://bilab.synology.me/smis/brand/adidas/ad...,abstract,[Leggings & Tights],adidas_zbt4o5v_
2,https://bilab.synology.me/smis/brand/reebok/re...,abstract,[Leggings & Tights],reebok_t_5ziwiu
3,https://bilab.synology.me/smis/brand/under_arm...,abstract,[Leggings & Tights],under_armour_u9a8s1te
4,https://bilab.synology.me/smis/brand/adidas/ad...,spots,[Jackets & Vests],adidas_lq_z8pdy


In [55]:
# # 檢查預測正確率
# pict_1000 = url_pattern_df['image_url'][:100]
# patt_1000 = url_pattern_df['pattern'][:100]
# cloth_1000 = url_pattern_df['clothing'][:100]


# corrects = 0
# for i in range(len(pict_1000)):
#     if i % 10 ==0:
#         print(i)
#     output = predict(pict_1000[i], "url", cloth_1000[i])
#     print(output, patt_1000[i])
#     if output == patt_1000[i]:
#         corrects += 1
# print(corrects/len(pict_1000))
    

0


<ipython-input-49-79fc5acfeea7>:36: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():


placement placement
abstract abstract
abstract abstract
geometric abstract
spots spots
abstract geometric
geometric geometric
abstract abstract
abstract solid
camouflage abstract
10
melange melange
geometric geometric
placement placement
camouflage abstract
melange melange
placement placement
abstract abstract
melange melange
placement placement
geometric melange
20
strips strips
geometric geometric
melange melange
melange melange
melange melange
melange melange
strips strips
geometric placement
melange melange
abstract abstract
30
melange placement
strips strips
melange melange
geometric abstract
camouflage abstract
melange melange
abstract abstract
solid geometric
melange melange
melange melange
40
placement placement
melange melange
strips strips
melange melange
melange melange
melange melange
melange melange
melange melange
strips strips
placement placement
50
strips strips
abstract abstract
melange melange
geometric geometric
abstract abstract
abstract abstract
abstract abstract
a